In [9]:
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language, RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

import os
from git import Repo

In [40]:
repo_path = "./bailey"

In [41]:
repo = Repo.clone_from("https://github.com/jardelkuhnen/baylei",to_path=repo_path)

In [42]:
loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes = [".java"],
    parser = LanguageParser(language=Language.JAVA, parser_threshold=500)
)

documents = loader.load()
len(documents)

69

In [43]:
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.JAVA, chunk_size = 2000, chunk_overlap = 200
)

texts = python_splitter.split_documents(documents)
len(texts)

98

In [44]:
os.environ['OPENAI_API_KEY'] = "token openai"

In [45]:
db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=()))

retriever = db.as_retriever(
    search_type = "mmr",
    search_kwargs= {"K": 8}
)

In [55]:
llm = ChatOpenAI(model_name="gpt-4o-mini", max_tokens = 500)

In [56]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um revisor de código experiente em Java. Forneça informações detalhadas sobre a revisão de código e sugestões de melhorias baseado no contexto fornecido abaixo \n\n {context}"
        ),
        (
            "user", "{input}"
        )
    ]
)

document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [54]:
response = retrieval_chain.invoke({"input": "Você pode revisar e sugerir melhorias para o código de ClientServiceAdapter"})

In [53]:
print(response['answer'])

Claro! Aqui estão algumas observações e sugestões de melhorias para o código da classe `ClientServiceAdapter`:

### 1. Importações não utilizadas
Certifique-se de que todas as importações são necessárias. Por exemplo, a classe `java.util.Optional` não é utilizada no código. Remova importações desnecessárias para manter o código limpo.

### 2. Tratamento de exceções
O código atualmente lança uma exceção personalizada `NotFoundException` quando um cliente não é encontrado. Seria interessante garantir que essa exceção esteja bem definida em um pacote apropriado. Além disso, considere adicionar tratamento de exceção específico para outros erros que possam ocorrer no processo.

### 3. Uso de `LocalDateTime.now()`
O uso de `LocalDateTime.now()` pode ser problemático em um ambiente de testes, pois ele depende do relógio do sistema. Utilize um serviço de relógio (clock) para facilitar testes e permitir o controle do tempo. Por exemplo:

```java
import java.time.Clock;

private final Clock cloc